In [99]:
# Dependencies
import pymongo
import datetime
import pandas as pd
import re

In [112]:
# Connect to database
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['carsguide_used_cars_australia']
collection = db.scraped_cars

# Create df
df = pd.DataFrame(list(collection.find()))
df.head()

,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
0,616017a57591e422f04df82e,Parkside Motor Co,2006 Mercedes-Benz C200 Kompressor,189474,Automatic,8990,Just arrived stunning C200 Kompressor Sport in...,https://www.carsguide.com.au/buy-a-car/all-new...,2006,Mercedes-Benz,"[C200, Kompressor, ]",WA,Maylands,USED Dealer ad,2021/10/08,0
1,616017a57591e422f04df82f,Great Deal Auto,2009 Ford Ranger XL,195565,Automatic,15500,We offer onsite finance through Australias lea...,https://www.carsguide.com.au/buy-a-car/all-new...,2009,Ford,"[Ranger, XL]",WA,Malaga,USED Dealer ad,2021/10/08,0
2,616017a57591e422f04df830,Fabcar,2004 Toyota Estima Aeras,84973,Automatic,29888,** VIEWINGS BY APPOINTMENT ONLY **** MOBILITY ...,https://www.carsguide.com.au/buy-a-car/all-new...,2004,Toyota,"[Estima, Aeras]",WA,Perth,USED Dealer ad,2021/10/08,0
3,616017a57591e422f04df831,Osborne Park Nissan used,2018 Dodge N/A Laramie Crew Cab SWB,106000,Automatic,149999,THOUSANDS SPEND ON UPGRADES!! heavy duty rear ...,https://www.carsguide.com.au/buy-a-car/all-new...,2018,Dodge,"[N/A, Laramie, Crew, Cab, SWB]",WA,Osborne Park,USED Dealer ad,2021/10/08,0
4,616017a57591e422f04df832,Subaru Wangara used,2015 Volkswagen Golf R,102001,Manual,39998,LIVE THE FULL HOT HATCH EXPERIENCE.. Tastefull...,https://www.carsguide.com.au/buy-a-car/all-new...,2015,Volkswagen,"[Golf, R]",WA,Wangara,USED Dealer ad,2021/10/08,0


In [140]:
# Clean Data
df = df[df.Price != 'POA']
df['Price'] = df.loc[:,"Price"].astype(float)
df = df[df.Price > 500]
df = df[df.Price < 900000]
df = df[df.Price != 1234]
df = df[df['Dealer Name'] != 'POA']
df = df[df['State Abrev'] != 'Hyundai']
df.head()

,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
0,616017a57591e422f04df82e,Parkside Motor Co,2006 Mercedes-Benz C200 Kompressor,189474,Automatic,"$8,990.00",Just arrived stunning C200 Kompressor Sport in...,https://www.carsguide.com.au/buy-a-car/all-new...,2006,Mercedes-Benz,"[C200, Kompressor, ]",WA,Maylands,USED Dealer ad,2021/10/08,0
1,616017a57591e422f04df82f,Great Deal Auto,2009 Ford Ranger XL,195565,Automatic,"$15,500.00",We offer onsite finance through Australias lea...,https://www.carsguide.com.au/buy-a-car/all-new...,2009,Ford,"[Ranger, XL]",WA,Malaga,USED Dealer ad,2021/10/08,0
2,616017a57591e422f04df830,Fabcar,2004 Toyota Estima Aeras,84973,Automatic,"$29,888.00",** VIEWINGS BY APPOINTMENT ONLY **** MOBILITY ...,https://www.carsguide.com.au/buy-a-car/all-new...,2004,Toyota,"[Estima, Aeras]",WA,Perth,USED Dealer ad,2021/10/08,0
3,616017a57591e422f04df831,Osborne Park Nissan used,2018 Dodge N/A Laramie Crew Cab SWB,106000,Automatic,"$149,999.00",THOUSANDS SPEND ON UPGRADES!! heavy duty rear ...,https://www.carsguide.com.au/buy-a-car/all-new...,2018,Dodge,"[N/A, Laramie, Crew, Cab, SWB]",WA,Osborne Park,USED Dealer ad,2021/10/08,0
4,616017a57591e422f04df832,Subaru Wangara used,2015 Volkswagen Golf R,102001,Manual,"$39,998.00",LIVE THE FULL HOT HATCH EXPERIENCE.. Tastefull...,https://www.carsguide.com.au/buy-a-car/all-new...,2015,Volkswagen,"[Golf, R]",WA,Wangara,USED Dealer ad,2021/10/08,0


In [ ]:
# Checking for anomolies and outliers
df.sort_values(by=['Price'])

In [141]:
# Number of listings per state
df['State Abrev'].value_counts()

VIC    12297
NSW    12215
QLD     9231
WA      8169
SA      3404
TAS     1109
ACT      779
NT       440
Name: State Abrev, dtype: int64

In [142]:
# Number of listings per Suburb
df['Suburb'].value_counts()

Victoria Park        1147
Minchinbury          1128
Wangara              1007
Bentley               891
Berwick               689
                     ... 
Ringwood East           1
Flying Fish Point       1
Kearns                  1
Runcorn                 1
Lane Cove West          1
Name: Suburb, Length: 1911, dtype: int64

In [145]:
# Work out average Price by State
pd.options.display.float_format = '${:,.2f}'.format
df_state_grouped = df.loc[:,['State Abrev', 'Suburb', 'Price','KMs', 'Comments']]
df_state_grouped = df_state_grouped.groupby(['State Abrev'])
df_state_grouped['Price'].mean().round()


State Abrev
ACT   $32,369.00
NSW   $31,040.00
NT    $33,310.00
QLD   $28,707.00
SA    $27,591.00
TAS   $30,616.00
VIC   $33,258.00
WA    $34,196.00
Name: Price, dtype: float64

In [139]:
# Work out average Price by Suburb
pd.options.display.float_format = '${:,.2f}'.format
df_state_grouped = df.loc[:,['State Abrev', 'Suburb', 'Price','KMs', 'Comments']]
df_state_grouped = df_state_grouped.groupby(['State Abrev','Suburb'])
df_state_grouped['Price'].mean().round()

State Abrev  Suburb   
ACT          Amaroo      $56,600.00
             Belconnen   $29,641.00
             Bonython    $36,500.00
             Braddon     $11,100.00
             Bruce       $10,000.00
                            ...    
WA           Winthrop    $32,000.00
             Woodvale    $19,833.00
             Yanchep     $11,500.00
             Yangebup    $11,429.00
             Yokine      $24,400.00
Name: Price, Length: 1986, dtype: float64